In [1]:
import pandas as pd

In [2]:
columns = ['id', 'text', 'label']

train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', names=columns, skiprows=1).dropna()
test_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t', names=columns, skiprows=1).dropna()

In [3]:
train_data.to_csv('C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis\\train_data.csv', index=False)
test_data.to_csv('C:\\Users\\admin\\jupyter\\pytorch\\Sentiment_analysis\\test_data.csv', index=False)

In [4]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
# torch.backends.cudnn.determinisitic = True

In [5]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

In [6]:
# TREC : torchtext에서 제공하는 질문 분류 데이터셋
# fine_grained : 더 세밀하게 나눈다? / coarse_grained : 큰 덩어리로 나눈다?
train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

In [7]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
len(train_data), len(valid_data), len(test_data)

(3816, 1636, 500)

In [9]:
vars(train_data[-3])

{'text': ['Who',
  'is',
  'section',
  'manager',
  'for',
  'guidance',
  'and',
  'control',
  'systems',
  'at',
  'JPL',
  '?'],
 'label': 'HUM'}

In [10]:
vars(train_data[-2])

{'text': ['What',
  'year',
  'did',
  'Jack',
  'Nicklaus',
  'join',
  'the',
  'Professional',
  'Golfers',
  'Association',
  'tour',
  '?'],
 'label': 'NUM'}

In [11]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors='glove.6B.100d', unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

- HUM for questions about humans(인간에 대한 질문?)
- ENTY for questions about entities(엔티티(실제?)에 대한 질문)
- DESC for questions asking you for a description(설명을 요구하는 질문)
- NUM for questions where the answer is numerical(답이 숫자인 질문)
- LOC for questions where the answer is a location(답이 위치(인덱스)인 질문)
- ABBR for questions asking about abbreviations(약어에 대해 묻는 질문)

In [12]:
print(LABEL.vocab.stoi)

defaultdict(None, {'HUM': 0, 'ENTY': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


In [13]:
BATCH_SIZE = 64

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = 'cpu')

In [14]:
next(iter(train_iterator)).text.shape

torch.Size([19, 64])

In [15]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1,
                                             out_channels=n_filters,
                                             kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [sent_len, batch_size]
        # 처음에 batch_first를 하지 않았기 때문에 여기서 재배치하지 않았나?
        text = text.permute(1,0)
        #text = [batch_size, sent_len]
        
        embedded = self.embedding(text)
        #embedded = [batch_size, sent_len, emb_dim]
        
        embedded = embedded.unsqueeze(1)
        #embedded = [batch_size, 1, sent_len, emb_dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conv_n = [batch_size, n_filters, sent_len - fs[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch_size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        #cat = [batch_size, n_filters * len(filter_size)]
        
        return self.fc(cat)

In [16]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [17]:
model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'이 모델은 {count_parameters(model):,}개의 훈련 가능한 파라미터가 있습니다.')

이 모델은 842,406개의 훈련 가능한 파라미터가 있습니다.


In [19]:
pretrain_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrain_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [ 0.1638,  0.6046,  1.0789,  ..., -0.3140,  0.1844,  0.3624],
        ...,
        [-0.3110, -0.3398,  1.0308,  ...,  0.5317,  0.2836, -0.0640],
        [ 0.0091,  0.2810,  0.7356,  ..., -0.7508,  0.8967, -0.7631],
        [ 0.4306,  1.2011,  0.0873,  ...,  0.8817,  0.3722,  0.3458]])

In [20]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [21]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

# CrossEntropyLoss : 다중 분류를 위한 손실함수
# BCEWithLogitsLoss : BCELoss + sigmoid layer
# BCELoss : 2개인 binary case인 경우에 쓰이는 함수(따로 activation function을 써주어야 함)
criterion = nn.CrossEntropyLoss()

# model = model.to(device)
# criterion = criterion.to(device)

In [22]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim=1, keepdim=True)
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [23]:
pred = torch.tensor([[5.0, 0.1, 0.3], [2.3, 3.1, 0.7]])
pred

tensor([[5.0000, 0.1000, 0.3000],
        [2.3000, 3.1000, 0.7000]])

In [24]:
pred.argmax(dim=1).shape, pred.argmax(dim=1, keepdim=True).shape, pred.argmax(dim=1, keepdim=True)

(torch.Size([2]),
 torch.Size([2, 1]),
 tensor([[0],
         [1]]))

In [25]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [26]:
def eval(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [28]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = eval(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.300 | Train Acc: 49.28%
	Valid Loss: 0.936 | Valid Acc: 64.93%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.875 | Train Acc: 67.28%
	Valid Loss: 0.753 | Valid Acc: 74.75%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.674 | Train Acc: 76.49%
	Valid Loss: 0.637 | Valid Acc: 76.91%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.498 | Train Acc: 83.26%
	Valid Loss: 0.549 | Valid Acc: 80.72%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.384 | Train Acc: 87.82%
	Valid Loss: 0.498 | Valid Acc: 82.47%


In [29]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = eval(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.409 | Test Acc: 85.53%


In [34]:
import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len=4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim=1)
    return max_preds.item()

In [35]:
q = 'The one who told me the truth is Inhwan Lee.'
pred_class = predict_class(model, q)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = HUM


In [36]:
q = 'The central part of Korea is called Gyung-gi do.'
pred_class = predict_class(model, q)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 4 = LOC
